In [2]:
%reload_ext autoreload
%autoreload 2
%cd llama3/llama

/home/tbai4/llama3/llama


/home/tbai4/llama3/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
from llama import Workflow, Llama

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

workflow = Workflow.build(
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=512*16,
    max_batch_size=1,
    model_parallel_size=1,
    max_nodes=100,
    max_parents=10
)

llama = Llama(workflow.model, workflow.tokenizer)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Loaded in 16.92 seconds


In [3]:
workflow.reset()

[generate_prompt, evaluate_prompt] = workflow.insert([
    {
        'messages': [
            {'role': 'system', 'content': 'You will be shown a math problem. Provide a high-level summary of how to solve the problem.'},
            {'role': 'user', 'content': 'Here is the problem: How many ways can you sit 10 different people around a round table?'}
        ],
        'parent_ids': [],
    },
    {
        'messages': [
            {'role': 'system', 'content': 'Given a math problem and numerous proposed solutions, echo back to the user all of the solutions.'},
        ],
        'parent_ids': []
    }
])

resp1, resp2 = workflow.insert([
    {
        'messages': [{'role': 'assistant', 'content': 'We can possibly use binomial coefficients to solve this problem via combinations.'}],
        'parent_ids': [generate_prompt['id']],
    },
    {
        'messages': [{'role': 'assistant', 'content': 'We can consider all 10 permutations and cancel out the rotational symmetry.'}],
        'parent_ids': [evaluate_prompt['id']],
    },
])

tokens, cached = workflow.step(
    tasks=[
        {
            'header': ('assistant', None),
            'parent_ids': [evaluate_prompt['id'], resp2['id'], resp1['id']],
        }
        for _ in range(4)
    ],
    compact=False,
    max_gen_len=32,
    temperature=1.0,
    top_p=1.0,
    seed=1,
    stateless=False
)

for i, output in enumerate(tokens):
    print(f'example #{i+1}')
    print(workflow.tokenizer.decode(output))
    print()

example #1
To solve the problem, we can count the number of ways to assign a role to each person. 

First, let's choose any of the 4

example #2
However, thinking about it, if we fix one element, the rest can be arranged in a circular manner, but since there are 9 numbers left to

example #3
However, we don't want to divide by the number of rotations. We can first fix one number as the first.

example #4
We know that there are (n-1)! ways to arrange the remaining elements in a circular permutation. In this case, n is the total number of



In [4]:
from llama.benchmark import benchmark_workflow, solve_workflow

In [5]:
# load data

import json
from pathlib import Path

def load_math_problems(root_dir, split, problem_types):
    problems = []
    root = Path(root_dir) / split
    
    for problem_type in problem_types:
        type_dir = root / problem_type
        if not type_dir.exists():
            continue
        for prob_file in type_dir.glob("*.json"):
            with open(prob_file) as f:
                problem = json.load(f)
                problems.append(problem)
    
    return problems

problems = load_math_problems('../../MATH', 'train', ['counting_and_probability'])

In [6]:
case = {
    'workflow': workflow,
    'problem': problems[1]['problem'],
    'branching_factor': 2,
    'voters': 5
}

results = benchmark_workflow(
    workflow_fn=solve_workflow,
    test_cases=[case],
    n_trials=3,
    output_dir='profile/tot'
)

Trial 1
Finished in 11.961941386805847
Memory usage: 1.38 GB
Trial 2
Finished in 14.78994380007498
Memory usage: 1.70 GB
Trial 3
Finished in 23.786685358034447
Memory usage: 3.61 GB

Profile for case 1:


IOStream.flush timed out


AttributeError: 'dict' object has no attribute 'append'

In [ ]:
results